### import


In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

import os


import re
import datetime
from datetime import datetime, timedelta

In [2]:
# проверка на работу в "google colab" или "локально"

if 'sample_data' in os.listdir():
    project_dir = r'/content/drive/My Drive/Colab Notebooks/module_4/'
    print('Обнаружена среда выполнения Google Colab.')
    print('project_dir =>', project_dir)
else:
    project_dir = ''
    print('НЕ обнаружена среда выполнения Google Colab. Выбран режим локальной работы.')

Обнаружена среда выполнения Google Colab.
project_dir => /content/drive/My Drive/Colab Notebooks/module_4/


In [3]:
RANDOM_SEED = 1314

### Функции обработки

### DATA


In [4]:
# Выведем листинг проектного каталога
for dirname, _, filenames in os.walk(project_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/content/drive/My Drive/Colab Notebooks/module_4/AAQ_flights.csv
/content/drive/My Drive/Colab Notebooks/module_4/AAQ_profit_model.pdf
/content/drive/My Drive/Colab Notebooks/module_4/u5_analysis_of_flights.ipynb


In [5]:
flight = pd.read_csv(project_dir + 'AAQ_flights.csv')
flight.sample(9)

,flight_id,sched_year,sched_month,sched_day,departure_airport,flight_no,arrival_airport,scheduled_departure,scheduled_arrival,delay_departure,status,flight_duration,city,timezone,actual_departure,actual_arrival,delay_arrival,aircraft_code,manufacturer,board_model,flights_range_km,class2,ticket_count2,amount_sold2,avg_price_class2,class1,ticket_count1,amount_sold1,avg_price_class1,total_amount
65,136869,2017,1,31,AAQ,PG0480,EGO,2017-01-31 12:25:00,2017-01-31 13:15:00,00:01:00,Arrived,00:50:00,Belgorod,Europe/Moscow,2017-01-31 12:26:00,2017-01-31 13:16:00,00:01:00,SU9,Sukhoi,Superjet-100,3000,Economy,85.0,538500.0,6335.0,Business,12.0,226800.0,18900.0,765300.0
7,136819,2017,1,4,AAQ,PG0480,EGO,2017-01-04 12:25:00,2017-01-04 13:15:00,00:04:00,Arrived,00:50:00,Belgorod,Europe/Moscow,2017-01-04 12:29:00,2017-01-04 13:19:00,00:04:00,SU9,Sukhoi,Superjet-100,3000,Economy,84.0,532200.0,6336.0,Business,12.0,226800.0,18900.0,759000.0
25,136316,2017,1,12,AAQ,PG0252,SVO,2017-01-12 13:05:00,2017-01-12 14:45:00,00:02:00,Arrived,01:40:00,Moscow,Europe/Moscow,2017-01-12 13:07:00,2017-01-12 14:47:00,00:02:00,733,Boeing,737-300,4200,Economy,118.0,1446800.0,12261.0,Business,12.0,439200.0,36600.0,1886000.0
61,136178,2017,1,29,AAQ,PG0252,SVO,2017-01-29 13:05:00,2017-01-29 14:45:00,00:05:00,Arrived,01:39:00,Moscow,Europe/Moscow,2017-01-29 13:10:00,2017-01-29 14:49:00,00:04:00,733,Boeing,737-300,4200,Economy,90.0,1105200.0,12280.0,Business,9.0,329400.0,36600.0,1434600.0
108,136403,2017,2,20,AAQ,PG0252,SVO,2017-02-20 13:05:00,2017-02-20 14:45:00,00:01:00,Arrived,01:40:00,Moscow,Europe/Moscow,2017-02-20 13:06:00,2017-02-20 14:46:00,00:01:00,733,Boeing,737-300,4200,Economy,117.0,1434600.0,12262.0,Business,12.0,439200.0,36600.0,1873800.0
53,136383,2017,1,25,AAQ,PG0252,SVO,2017-01-25 13:05:00,2017-01-25 14:45:00,00:04:00,Arrived,01:42:00,Moscow,Europe/Moscow,2017-01-25 13:09:00,2017-01-25 14:51:00,00:06:00,733,Boeing,737-300,4200,Economy,98.0,1200400.0,12249.0,Business,8.0,292800.0,36600.0,1493200.0
81,136284,2017,2,7,AAQ,PG0252,SVO,2017-02-07 13:05:00,2017-02-07 14:45:00,00:02:00,Arrived,01:41:00,Moscow,Europe/Moscow,2017-02-07 13:07:00,2017-02-07 14:48:00,00:03:00,733,Boeing,737-300,4200,Economy,107.0,1311400.0,12256.0,Business,10.0,366000.0,36600.0,1677400.0
58,136678,2017,1,28,AAQ,PG0480,EGO,2017-01-28 12:25:00,2017-01-28 13:15:00,00:02:00,Arrived,00:50:00,Belgorod,Europe/Moscow,2017-01-28 12:27:00,2017-01-28 13:17:00,00:02:00,SU9,Sukhoi,Superjet-100,3000,Economy,70.0,442200.0,6317.0,Business,10.0,189000.0,18900.0,631200.0
49,136560,2017,1,24,AAQ,PG0194,NOZ,2017-01-24 09:10:00,2017-01-24 14:15:00,00:03:00,Arrived,05:05:00,Novokuznetsk,Asia/Novokuznetsk,2017-01-24 09:13:00,2017-01-24 14:18:00,00:03:00,733,Boeing,737-300,4200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
flight.columns

Index(['flight_id', 'sched_year', 'sched_month', 'sched_day',
       'departure_airport', 'flight_no', 'arrival_airport',
       'scheduled_departure', 'scheduled_arrival', 'delay_departure', 'status',
       'flight_duration', 'city', 'timezone', 'actual_departure',
       'actual_arrival', 'delay_arrival', 'aircraft_code', 'manufacturer',
       'board_model', 'flights_range_km', 'class2', 'ticket_count2',
       'amount_sold2', 'avg_price_class2', 'class1', 'ticket_count1',
       'amount_sold1', 'avg_price_class1', 'total_amount'],
      dtype='object')

In [7]:
len(flight.columns)

30

In [8]:
# количество направлений
flight['city'].value_counts()

Moscow          59
Belgorod        59
Novokuznetsk     9
Name: city, dtype: int64

In [9]:
# количество уникальных рейсов
flight['flight_no'].value_counts()

PG0252    59
PG0480    59
PG0194     9
Name: flight_no, dtype: int64

In [10]:
flight['board_model'].value_counts()

737-300         68
Superjet-100    59
Name: board_model, dtype: int64

In [11]:
flight['flight_duration'].value_counts().sort_values(ascending=False)

00:50:00    34
01:40:00    27
01:39:00    17
00:51:00    15
00:49:00    10
01:41:00     8
01:38:00     4
05:10:00     2
01:42:00     2
05:04:00     1
01:43:00     1
05:11:00     1
05:12:00     1
05:05:00     1
04:59:00     1
05:02:00     1
05:07:00     1
Name: flight_duration, dtype: int64


[Цены на авиатопливо в 2017](https://favt.gov.ru/dejatelnost-ajeroporty-i-ajerodromy-ceny-na-aviagsm-statistika-aviagsm/?id=3241)

Анапа (будем считать, что все заправки проводились в этом городе)
*  декабрь: 47101 руб\тонна
*  январь: 41435 руб\тонна
*  февраль: 39553 руб\тонна



In [12]:
# расход топлива за 1 час полета в кг
fuel_consumption = {'SU9': 1.7, '733': 2.4}

# стоимость топлива за 1000 кг
fuel_cost_2017 = {12: 47101, 1: 41435, 2: 39553}

# количество мест в самолете по классам обслуживания
cabin_size = {'SU9': {'Economy': 85, 'Business':12 },
              '733': {'Economy': 118, 'Business':12 }}

In [13]:
cabin_size['SU9']['Economy'], cabin_size['SU9']['Business']

(85, 12)

In [36]:
cabin_size['733']['Economy'], cabin_size['SU9']['Business']

(118, 12)

In [37]:
flight['flight_hour'] = flight['flight_duration'].apply(lambda x: re.findall(r"\d+", str(x)))
flight['flight_hour'] = flight['flight_hour'].apply(lambda x: round(float(x[0]) + float(x[1])/60, 3))

In [39]:
# Далее заполняем пропуски
flight.fillna(0, inplace=True)

In [40]:
flight['flight_costs'] = flight.apply(lambda row: row['flight_hour'] * fuel_cost_2017[row['sched_month']] * fuel_consumption[row['aircraft_code']], axis=1 )

In [41]:
flight['flight_profit'] = flight['total_amount'] - flight['flight_costs']

flight['flight_profit'] = flight['flight_profit'].apply(lambda x: int(x))

flight['total_amount'] = flight['total_amount'].apply(lambda x: int(x))

flight['flight_profit'] = flight['flight_profit'].apply(lambda x: int(x))

In [42]:
# проблемы с длительностью 'sched_month', 'sched_month',
fl = flight[['flight_id',  'flight_no', 'flight_duration', 'city',
        'aircraft_code', 'total_amount', 'flight_hour', 'flight_costs', 'flight_profit' ]]

fl.sample(15, random_state=RANDOM_SEED)

,flight_id,flight_no,flight_duration,city,aircraft_code,total_amount,flight_hour,flight_costs,flight_profit
93,136130,PG0252,01:39:00,Moscow,733,1556600,1.650,156629.8800,1399970
106,136320,PG0252,01:40:00,Moscow,733,1544400,1.667,158243.6424,1386156
100,136268,PG0252,01:41:00,Moscow,733,1676200,1.683,159762.4776,1516437
9,136645,PG0480,00:51:00,Belgorod,SU9,633000,0.850,59873.5750,573126
109,136511,PG0194,05:10:00,Novokuznetsk,733,0,5.167,490488.8424,-490488
63,136226,PG0252,01:40:00,Moscow,733,1886000,1.667,165773.1480,1720226
61,136178,PG0252,01:39:00,Moscow,733,1434600,1.650,164082.6000,1270517
53,136383,PG0252,01:42:00,Moscow,733,1493200,1.700,169054.8000,1324145
71,136672,PG0480,00:50:00,Belgorod,SU9,759000,0.833,56011.0033,702988
12,136209,PG0252,01:40:00,Moscow,733,1615200,1.667,165773.1480,1449426


In [43]:
len(fl.columns), fl.columns

(9,
 Index(['flight_id', 'flight_no', 'flight_duration', 'city', 'aircraft_code',
        'total_amount', 'flight_hour', 'flight_costs', 'flight_profit'],
       dtype='object'))

In [63]:
fl.groupby(['city'])['total_amount'].sum()

city
Belgorod        41915700
Moscow          96912000
Novokuznetsk           0
Name: total_amount, dtype: int64

In [97]:
fl[fl['total_amount'] <= 0].groupby(['city'])['flight_no'].count()


Series([], Name: flight_no, dtype: int64)

In [73]:
fl.query('total_amount <= 0')

,flight_id,flight_no,flight_duration,city,aircraft_code,total_amount,flight_hour,flight_costs,flight_profit
4,136546,PG0194,05:02:00,Novokuznetsk,733,0,5.033,500501.6520,-500501
19,136523,PG0194,05:12:00,Novokuznetsk,733,0,5.200,517108.8000,-517108
34,136540,PG0194,05:07:00,Novokuznetsk,733,0,5.117,508854.9480,-508854
49,136560,PG0194,05:05:00,Novokuznetsk,733,0,5.083,505473.8520,-505473
64,136544,PG0194,05:04:00,Novokuznetsk,733,0,5.067,503882.7480,-503882
79,136514,PG0194,04:59:00,Novokuznetsk,733,0,4.983,473022.2376,-473022
94,136567,PG0194,05:11:00,Novokuznetsk,733,0,5.183,492007.6776,-492007
109,136511,PG0194,05:10:00,Novokuznetsk,733,0,5.167,490488.8424,-490488
124,136513,PG0194,05:10:00,Novokuznetsk,733,0,5.167,490488.8424,-490488


In [74]:
fl = fl.query('total_amount > 0')
fl.shape

In [89]:
# fl['flight_profit'].idxmin(axis="columns")

fl.iloc[fl['flight_profit'].idxmin()]

flight_id            136900
flight_no            PG0480
flight_duration    00:51:00
city               Belgorod
aircraft_code           SU9
total_amount         765300
flight_hour            0.85
flight_costs        57154.1
flight_profit        708145
Name: 67, dtype: object

In [78]:
fl['flight_profit'].min()

473450

In [79]:
fl['flight_profit'].max()

1720226

In [99]:
fl.groupby('flight_no')['flight_profit'].sum()

flight_no
PG0252    87360076
PG0480    38521815
Name: flight_profit, dtype: int64